In [1]:
import pandas as pd
import pyspark
import pyspark.sql.types


from pyspark.sql import SparkSession, SQLContext
from pyspark import SparkConf
from pyspark.sql.functions import sum, avg, count, first
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [2]:
baskets = pd.read_csv('../../data/02_intermediate/baskets_spark.csv')

In [3]:
baskets.drop(columns=['Unnamed: 0'], inplace=True)

Get a sub-sample to test with spark 

In [4]:
conf = SparkConf().setMaster('local').setAppName('autoencoder_recommender_wide_user_record_maker').set("spark.sql.pivotMaxValues", 99999)

In [5]:
sc = pyspark.SparkContext(conf=conf)

In [6]:
spark = SparkSession.builder.getOrCreate()

In [7]:
sqlContext = SQLContext(sc, spark)

In [8]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [10]:
df_full = sqlContext.read.format('com.databricks.spark.csv').options(header='true', 
                                                                inferschema='true').load('../../data/02_intermediate/baskets_spark.csv')

In [11]:
df_full.show()

+---+--------+--------------------+-------+-------------+
|_c0|order_id|        product_name|user_id|product_count|
+---+--------+--------------------+-------+-------------+
|  0|       2|  Organic Egg Whites| 202279|            1|
|  1|       2|Michigan Organic ...| 202279|            1|
|  2|       2|       Garlic Powder| 202279|            1|
|  3|       2|      Coconut Butter| 202279|            1|
|  4|       2|   Natural Sweetener| 202279|            1|
|  5|       2|             Carrots| 202279|            1|
|  6|       2|Original Unflavor...| 202279|            1|
|  7|       2|All Natural No St...| 202279|            1|
|  8|       2|Classic Blend Col...| 202279|            1|
|  9|       3|Total 2% with Str...| 205970|            1|
| 10|       3|Unsweetened Almon...| 205970|            1|
| 11|       3|              Lemons| 205970|            1|
| 12|       3|Organic Baby Spinach| 205970|            1|
| 13|       3|Unsweetened Choco...| 205970|            1|
| 14|       3|

In [12]:
df_full.select("user_id", "product_name", "product_count").show()

+-------+--------------------+-------------+
|user_id|        product_name|product_count|
+-------+--------------------+-------------+
| 202279|  Organic Egg Whites|            1|
| 202279|Michigan Organic ...|            1|
| 202279|       Garlic Powder|            1|
| 202279|      Coconut Butter|            1|
| 202279|   Natural Sweetener|            1|
| 202279|             Carrots|            1|
| 202279|Original Unflavor...|            1|
| 202279|All Natural No St...|            1|
| 202279|Classic Blend Col...|            1|
| 205970|Total 2% with Str...|            1|
| 205970|Unsweetened Almon...|            1|
| 205970|              Lemons|            1|
| 205970|Organic Baby Spinach|            1|
| 205970|Unsweetened Choco...|            1|
| 205970| Organic Ginger Root|            1|
| 205970|Air Chilled Organ...|            1|
| 205970|Organic Ezekiel 4...|            1|
| 178520|Plain Pre-Sliced ...|            1|
| 178520|Honey/Lemon Cough...|            1|
| 178520|C

In [13]:
df_full.groupBy('product_name').count().show()

+--------------------+-----+
|        product_name|count|
+--------------------+-----+
|G Series Lime Cuc...|  217|
|XL Emerald White ...|15740|
|Cherry Garcia Ice...| 4283|
|Real Aged Cheddar...| 3262|
|Light Wisconsin S...|   63|
|Organic Cinnamon ...| 8073|
|Baby Food Stage 2...| 4460|
|Simply Stock Orga...| 2047|
|Enlightened Organ...| 7007|
|Organic White Win...| 1121|
|   Sliced Cooked Ham|  433|
|BoomChocoBoom Glu...|  499|
|from Concentrate ...|   81|
|  Premium Epsom Salt| 4228|
|Chocolate Chip Mu...|  532|
|Vegan Hickory & S...| 1512|
|Warm & Crunchy Ho...|   79|
| Organic Heavy Cream|  822|
|Grilled Chicken A...|  250|
|Chai Latte K-Cup ...|  164|
+--------------------+-----+
only showing top 20 rows



There are 49,677 unique products in the dataset

In [16]:
from pyspark.sql.functions import first

In [ ]:
(df_full
    .groupby(df_full.user_id, df_full.product_name)
    .pivot("product_name")
    .agg(first("product_count"))
    .show())

In [ ]:
df_full.select("product_name").distinct().count()

In [15]:
df_full.groupBy("user_id", "product_name").count().pivot("product_name").show()

AttributeError: 'DataFrame' object has no attribute 'pivot'

```python
basket_matrix = (baskets.groupby(['user_id', 'product_name'])['product_count']
          .sum().unstack().reset_index().fillna(0)
          .set_index('order_id'))